In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
import gensim.models as genmodels
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
df=pd.read_csv('medium_summary.csv')

In [4]:
df.head()

,article_url,best_of,title,summary,image_count,link_count,blockquote_count,publication_date,reading_time,clap_count,unique_clap_count,comment_count,author_url,author_name,publication_url,publication_name,codeblock_count,code_count,all_codes_list
0,https://medium.com/towards-data-science/unders...,All time,Understanding AUC - ROC Curve,[Image 1] (Image courtesy: My Photoshopped C...,13,4,2,"Jun 27, 2018",5,15949,4149,76,https://medium.com/@narkhedesarang?source=---t...,Sarang Narkhede,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0,['[]']
1,https://medium.com/towards-data-science/buildi...,All time,"Building A Logistic Regression in Python, Step...",Photo Credit: Scikit-Learn Logistic Regressio...,23,10,0,"Sep 29, 2017",9,11829,2671,133,https://actsusanli.medium.com/?source=---three...,Susan Li,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,22,128,"['import pandas as pd', 'import numpy as np', ..."
2,https://medium.com/towards-data-science/pca-us...,All time,PCA using Python (scikit-learn),Original image (left) with Different Amounts ...,8,14,0,"Dec 5, 2017",8,7248,1624,36,https://medium.com/@GalarnykMichael?source=---...,Michael Galarnyk,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,18,47,"['import pandas as pd', 'url = ""https://archiv..."
3,https://medium.com/towards-data-science/train-...,All time,Train/Test Split and Cross Validation in Python,Hi everyone! After my last post on linear reg...,6,14,0,"May 17, 2017",9,6597,1718,49,https://medium.com/@adi.bronshtein?source=---t...,Adi Bronshtein,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,47,"['import pandas as pd', 'from sklearn import d..."
4,https://medium.com/towards-data-science/the-5-...,All time,The 5 Clustering Algorithms Data Scientists Ne...,Want to be inspired? Come join my Super Quote...,8,1,1,"Feb 6, 2018",11,21321,4502,54,https://medium.com/@george.seif94?source=---th...,George Seif,https://towardsdatascience.com/?source=post_pa...,Towards Data Science,0,0,['[]']


In [5]:
data = df.summary.values.tolist()
# Convert to list 
data = df.summary.values.tolist()  
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]
#Remove \n
data = [re.sub("\n", "", sent) for sent in data]
data = [re.sub("(Image courtesy: )", "", sent) for sent in data]
#Remove links
data = [re.sub(r'http\S+', '', sent) for sent in data]
print(data[:1])

<>:5: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \s
C:\Users\colee\AppData\Local\Temp\ipykernel_15936\2013857722.py:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
C:\Users\colee\AppData\Local\Temp\ipykernel_15936\2013857722.py:7: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


[' [Image 1] (My Photoshopped Collection) In Machine Learning, performance measurement is an essential task. So when it comes to a classification problem, we can count on an AUC - ROC Curve. When we need to check or visualize the performance of the multi-class classification problem, we use the AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve. It is one of the most important evaluation metrics for checking any classification model’s performance. It is also written as AUROC (Area Under the Receiver Operating Characteristics) Note: For better understanding, I suggest you read my article about Confusion Matrix. This blog aims to answer the following questions: 1. What is the AUC - ROC Curve? 2. Defining terms used in AUC and ROC Curve. 3. How to speculate the performance of the model? 4. Relation between Sensitivity, Specificity, FPR, and Threshold. 5. How to use AUC - ROC curve for the multiclass model? What is the AUC - ROC Curve? AUC - ROC curve is a performanc

In [6]:
# #testing the pipeline (to remove once data is here)
# df=pd.read_json('newsgroups.json')
# print(df.target_names.unique())
# df.head()
# # Convert to list 
# data = df.content.values.tolist()

# # Remove Emails 
# data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# data = [re.sub('\s+', ' ', sent) for sent in data]  
# # Remove distracting single quotes 
# data = [re.sub("\'", "", sent) for sent in data]  
# pprint(data[:1])

In [7]:
# Loeading Cleaned Dataset


### Tokenization

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:1]) 

[['image', 'my', 'photoshopped', 'collection', 'in', 'machine', 'learning', 'performance', 'measurement', 'is', 'an', 'essential', 'task', 'so', 'when', 'it', 'comes', 'to', 'classification', 'problem', 'we', 'can', 'count', 'on', 'an', 'auc', 'roc', 'curve', 'when', 'we', 'need', 'to', 'check', 'or', 'visualize', 'the', 'performance', 'of', 'the', 'multi', 'class', 'classification', 'problem', 'we', 'use', 'the', 'auc', 'area', 'under', 'the', 'curve', 'roc', 'receiver', 'operating', 'characteristics', 'curve', 'it', 'is', 'one', 'of', 'the', 'most', 'important', 'evaluation', 'metrics', 'for', 'checking', 'any', 'classification', 'model', 'performance', 'it', 'is', 'also', 'written', 'as', 'auroc', 'area', 'under', 'the', 'receiver', 'operating', 'characteristics', 'note', 'for', 'better', 'understanding', 'suggest', 'you', 'read', 'my', 'article', 'about', 'confusion', 'matrix', 'this', 'blog', 'aims', 'to', 'answer', 'the', 'following', 'questions', 'what', 'is', 'the', 'auc', 'roc

### Building the bigram and trigram Models

In [9]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)


In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['image', 'my_photoshopped', 'collection', 'in', 'machine', 'learning', 'performance', 'measurement', 'is', 'an', 'essential', 'task', 'so', 'when', 'it', 'comes', 'to', 'classification', 'problem', 'we', 'can', 'count', 'on', 'an', 'auc_roc_curve', 'when', 'we', 'need', 'to', 'check', 'or', 'visualize', 'the', 'performance', 'of', 'the', 'multi', 'class', 'classification', 'problem', 'we', 'use', 'the', 'auc', 'area_under', 'the', 'curve', 'roc', 'receiver_operating', 'characteristics', 'curve', 'it', 'is', 'one', 'of', 'the', 'most', 'important', 'evaluation', 'metrics', 'for', 'checking', 'any', 'classification', 'model', 'performance', 'it', 'is', 'also', 'written', 'as', 'auroc', 'area_under', 'the', 'receiver_operating', 'characteristics', 'note', 'for', 'better', 'understanding', 'suggest', 'you', 'read', 'my', 'article', 'about', 'confusion_matrix', 'this', 'blog', 'aims', 'to', 'answer', 'the', 'following', 'questions', 'what', 'is', 'the', 'auc_roc_curve', 'defining', 'terms'

### Define function for stopwords, bigrams, trigrams and lemmatization

In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_nostops)


In [13]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_bi = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_tri = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized_bi[:1])

C:\Users\colee\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


[['image', 'photoshopped_collection', 'machine', 'learn', 'performance', 'measurement', 'essential', 'task', 'come', 'classification', 'problem', 'count', 'curve', 'check', 'performance', 'class', 'classification', 'problem', 'auc', 'area', 'curve', 'characteristic', 'curve', 'important', 'evaluation', 'metric', 'check', 'classification', 'model', 'performance', 'also', 'write', 'auroc', 'area', 'receiver_operate', 'characteristic', 'note', 'well', 'understanding', 'suggest', 'read', 'article', 'blog', 'aim', 'answer', 'follow', 'question', 'curve', 'define', 'term', 'use', 'curve', 'speculate', 'performance', 'model', 'relation', 'sensitivity', 'specificity', 'fpr', 'threshold', 'multiclass', 'performance', 'measurement', 'classification', 'problem', 'various', 'threshold', 'setting', 'curve', 'auc', 'represent', 'degree', 'measure', 'separability', 'tell', 'much', 'model', 'capable', 'distinguishing', 'class', 'high', 'auc', 'well', 'model', 'predict', 'class', 'class', 'analogy', 'h

In [14]:
id2word_bi = corpora.Dictionary(data_lemmatized_bi)  
id2word_tri = corpora.Dictionary(data_lemmatized_tri)  

In [15]:
texts_bi = data_lemmatized_bi
texts_tri = data_lemmatized_tri

In [16]:
# Term Document Frequency 
corpus_bi = [id2word_bi.doc2bow(text) for text in texts_bi]  
corpus_tri = [id2word_tri.doc2bow(text) for text in texts_tri]  

In [17]:
# View 
print(corpus_bi[:1])

[[(0, 2), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 10), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 2), (17, 1), (18, 2), (19, 2), (20, 1), (21, 20), (22, 4), (23, 1), (24, 2), (25, 1), (26, 1), (27, 13), (28, 3), (29, 2), (30, 1), (31, 1), (32, 4), (33, 4), (34, 1), (35, 4), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 7), (47, 3), (48, 2), (49, 1), (50, 1), (51, 1), (52, 3), (53, 1), (54, 2), (55, 9), (56, 1), (57, 5), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 7), (68, 4), (69, 2), (70, 1), (71, 1), (72, 1), (73, 16), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 6), (80, 2), (81, 2), (82, 1), (83, 2), (84, 3), (85, 1), (86, 6), (87, 6), (88, 3), (89, 1), (90, 7), (91, 2), (92, 3), (93, 2), (94, 3), (95, 1), (96, 2), (97, 1), (98, 1), (99, 2), (100, 1), (101, 1), (102, 2), (103, 1), (104, 2), (105, 1), (106, 1), (107, 2), (108, 9), (109, 4), (110

In [18]:
[[(id2word_bi[id], freq) for id, freq in cp] for cp in corpus_bi[:1]]
[[(id2word_tri[id], freq) for id, freq in cp] for cp in corpus_tri[:1]]

[[('able', 2),
  ('actually', 1),
  ('aim', 1),
  ('also', 2),
  ('always', 1),
  ('analogy', 1),
  ('answer', 1),
  ('approximately', 2),
  ('area', 2),
  ('article', 1),
  ('auc', 10),
  ('auc_roc_curve', 10),
  ('auroc', 1),
  ('axis', 2),
  ('bad', 2),
  ('blog', 1),
  ('capable', 1),
  ('capacity', 2),
  ('chance', 1),
  ('characteristic', 2),
  ('check', 2),
  ('clap', 1),
  ('class', 20),
  ('classification', 4),
  ('classified', 1),
  ('classify', 2),
  ('come', 1),
  ('count', 1),
  ('curve', 7),
  ('decrease', 3),
  ('define', 2),
  ('degree', 1),
  ('discrimination', 1),
  ('disease', 4),
  ('distinguish', 4),
  ('distinguishing', 1),
  ('distribution', 4),
  ('error', 1),
  ('essential', 1),
  ('evaluation', 1),
  ('exactly', 1),
  ('example', 1),
  ('excellent', 1),
  ('extra', 1),
  ('facebook', 1),
  ('fact', 1),
  ('follow', 1),
  ('fpr', 7),
  ('get', 3),
  ('give', 2),
  ('good', 1),
  ('green', 1),
  ('helpful', 1),
  ('high', 3),
  ('hope', 1),
  ('ideal', 2),
  ('i

### LSI & LDA Model

In [22]:
def model_competition(corpus,id2word,data_lemmatized):
    random_state = 100
    for_framing = []
    
    algo_names = ["LDA Model","LSI Model",]
    algorithms = [genmodels.ldamodel.LdaModel,genmodels.lsimodel.
                  LsiModel]
    
    for algo_name, algorithm in zip(algo_names, algorithms):
        for k in range(1, 11):
            if algo_name == 'LDA Model':
                model = algorithm(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
                perplexity = model.log_perplexity(corpus)
                coherence_model = CoherenceModel(model=model, texts=data_lemmatized, 
                               dictionary=id2word, coherence='c_v')
                coherence = coherence_model.get_coherence()
            if algo_name == 'LSI Model':
                model = algorithm(corpus=corpus,
                                  id2word=id2word,
                                  num_topics=k,
                                  chunksize=100)
        
                coherence_model = CoherenceModel(model=model, texts=data_lemmatized, 
                               dictionary=id2word, coherence='c_v')
                coherence = coherence_model.get_coherence()
        
            data_row = {
                'model': algo_name,
                'topic count' : k,
                'coherence' : coherence
            }
            for_framing.append(data_row)
        
    return pd.DataFrame(for_framing)
        
        
    

In [ ]:
bigram_dataframe = model_competition(corpus_bi,id2word_bi,data_lemmatized_bi)

In [58]:
bigram_dataframe.sort_values(by=['coherence'],ascending=False).head(1)

,model,topic count,coherence
8,LDA Model,9,0.604526


In [59]:
trigram_dataframe = model_competition(corpus_tri,id2word_tri,data_lemmatized_tri)

In [60]:
trigram_dataframe.sort_values(by=['coherence'],ascending=False).head(1)

,model,topic count,coherence
8,LDA Model,9,0.604363


# HDP

In [19]:
hdpmodel =  gensim.models.hdpmodel.HdpModel(corpus=corpus_bi, id2word=id2word_bi)
coherence_model_hdpmodel = CoherenceModel(model=hdpmodel, texts=data_lemmatized_bi, dictionary=id2word_bi, coherence='c_v')
coherence_hdpmodel = coherence_model_hdpmodel.get_coherence()
print('\nCoherence Score: ', coherence_hdpmodel)


Coherence Score:  0.543526652939898


In [21]:
hdpmodel.show_topics()

[(0,
  '0.015*datum + 0.010*model + 0.009*use + 0.005*data + 0.005*time + 0.005*learn + 0.005*make + 0.005*also + 0.004*image + 0.004*dataset + 0.004*need + 0.004*create + 0.004*value + 0.004*learning + 0.004*get + 0.004*function + 0.004*feature + 0.004*code + 0.004*work + 0.003*example'),
 (1,
  '0.019*datum + 0.009*model + 0.009*data + 0.007*use + 0.007*science + 0.006*learn + 0.006*work + 0.005*time + 0.005*value + 0.004*get + 0.004*make + 0.004*learning + 0.004*project + 0.004*machine + 0.004*good + 0.004*well + 0.004*need + 0.003*find + 0.003*also + 0.003*feature'),
 (2,
  '0.014*datum + 0.010*model + 0.009*use + 0.006*learn + 0.006*learning + 0.006*data + 0.005*function + 0.005*value + 0.005*time + 0.005*image + 0.004*create + 0.004*feature + 0.004*also + 0.004*dataset + 0.004*let + 0.004*make + 0.004*work + 0.004*see + 0.003*need + 0.003*machine'),
 (3,
  '0.009*datum + 0.007*use + 0.006*time + 0.006*code + 0.005*value + 0.004*function + 0.004*create + 0.004*need + 0.004*get + 0

In [22]:
hdpmodel_tri =  gensim.models.hdpmodel.HdpModel(corpus=corpus_tri, id2word=id2word_tri)
coherence_model_hdpmodel_tri = CoherenceModel(model=hdpmodel_tri, texts=data_lemmatized_tri, dictionary=id2word_tri, coherence='c_v')
coherence_hdpmodel_tri = coherence_model_hdpmodel_tri.get_coherence()
print('\nCoherence Score: ', coherence_hdpmodel_tri)


Coherence Score:  0.5348782847089283


In [23]:
hdpmodel_tri.show_topics()

[(0,
  '0.017*datum + 0.010*model + 0.009*use + 0.007*data + 0.006*time + 0.006*learn + 0.005*value + 0.005*learning + 0.004*make + 0.004*work + 0.004*also + 0.004*code + 0.004*image + 0.004*function + 0.004*need + 0.004*get + 0.004*science + 0.004*example + 0.004*well + 0.004*feature'),
 (1,
  '0.008*model + 0.008*use + 0.007*datum + 0.006*python + 0.005*dataset + 0.005*user + 0.005*function + 0.005*create + 0.004*new + 0.004*project + 0.004*method + 0.004*want + 0.004*get + 0.004*see + 0.004*time + 0.004*need + 0.004*make + 0.003*also + 0.003*learn + 0.003*work'),
 (2,
  '0.011*model + 0.010*datum + 0.007*use + 0.005*value + 0.005*image + 0.004*word + 0.004*make + 0.004*work + 0.004*data + 0.004*training + 0.003*time + 0.003*learning + 0.003*learn + 0.003*well + 0.003*also + 0.003*feature + 0.003*number + 0.003*set + 0.003*good + 0.003*dataset'),
 (3,
  '0.015*datum + 0.008*data + 0.007*learn + 0.006*science + 0.005*work + 0.004*job + 0.004*get + 0.004*need + 0.004*use + 0.004*help +